In [11]:
import pandas as pd
file_path = '/content/all_kindle_review.csv'
kindle_reviews = pd.read_csv(file_path)

kindle_reviews['reviewText'] = kindle_reviews['reviewText'].str.lower()

kindle_reviews.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"jace rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,great short read. i didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,i'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,aggie is angela lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,i did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [12]:
def rating(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

kindle_reviews['sentiment'] = kindle_reviews['rating'].apply(rating)

kindle_reviews_cleaned = kindle_reviews[['reviewText', 'sentiment']].dropna()

sentiment_distribution = kindle_reviews_cleaned['sentiment'].value_counts()

kindle_reviews_cleaned.head(), sentiment_distribution


(                                          reviewText sentiment
 0  jace rankin may be short, but he's nothing to ...   neutral
 1  great short read.  i didn't want to put it dow...  positive
 2  i'll start by saying this is the first of four...   neutral
 3  aggie is angela lansbury who carries pocketboo...   neutral
 4  i did not expect this type of book to be in li...  positive,
 sentiment
 positive    6000
 negative    4000
 neutral     2000
 Name: count, dtype: int64)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = kindle_reviews_cleaned['reviewText']
y = kindle_reviews_cleaned['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
y

,sentiment
0,neutral
1,positive
2,neutral
3,neutral
4,positive
...,...
11995,positive
11996,positive
11997,neutral
11998,negative


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)


y_pred = model.predict(X_test_tfidf)

In [18]:
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(report)
print(accuracy)


              precision    recall  f1-score   support

    negative       0.78      0.82      0.80       800
     neutral       0.48      0.18      0.26       400
    positive       0.78      0.92      0.84      1200

    accuracy                           0.76      2400
   macro avg       0.68      0.64      0.63      2400
weighted avg       0.73      0.76      0.73      2400

0.7616666666666667
